In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [4]:
# csv file with paths to all embeddings
df = pd.read_csv("/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/2d_audio_text_paths.csv")

df.head()

,split,PTSD_severity,PTSD_label,gender,audio,text
0,train,22.0,0,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
1,test,23.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
2,train,19.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
3,train,67.0,1,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
4,dev,39.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...


In [5]:
majority_gender = 'male'
minority_gender = 'female'
# DataFrame for the majority gender (Male)
majority_df = df[df['gender'] == majority_gender]

# DataFrame for the minority gender (Female)
minority_df = df[df['gender'] == minority_gender]

# Count of minority instances
minority_count = len(minority_df)
print(minority_count)

105


## Undersampling

In [6]:
# Calculate the proportion of each split ('train', 'test', 'dev', etc.) within the majority gender
split_ratios = majority_df['split'].value_counts(normalize=True)
# Initialize a dictionary to store the number of samples per split
samples_per_split = {}

# Calculate the number of samples for each split
for split in split_ratios.index:
    samples_per_split[split] = int(split_ratios[split] * minority_count)

# Adjust the total number of samples to match the minority count
total_samples = sum(samples_per_split.values())
difference = minority_count - total_samples

# Adjust the largest split to account for rounding differences
if difference != 0:
    largest_split = split_ratios.idxmax()
    samples_per_split[largest_split] += difference

In [7]:
# Initialize an empty DataFrame to store sampled data
sampled_majority = pd.DataFrame()

# Randomly sample the required number of instances for each split
for split, n_samples in samples_per_split.items():
    split_data = majority_df[majority_df['split'] == split]
    sampled_split = split_data.sample(n=n_samples, random_state=24)
    sampled_majority = pd.concat([sampled_majority, sampled_split], ignore_index=True)

# Combine the sampled majority data with all of the minority data
balanced_df = pd.concat([sampled_majority, minority_df], ignore_index=True)

In [9]:
# Shuffle the combined DataFrame to mix the data
balanced_df = shuffle(balanced_df, random_state=42).reset_index(drop=True)

# Check that the dataset is balanced
print("Gender Counts:\n", balanced_df['gender'].value_counts())

# Check that the split ratios are maintained within each gender
print("\nSplit Counts:\n", balanced_df.groupby('gender')['split'].value_counts())

undersampled_df = balanced_df
features_csv = pd.read_csv("/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/2d_audio_text_paths.csv")

undersampled_df.to_csv("/home/hice1/mbibars3/scratch/vlm-debiasing/data/e-daic/undersampled_embeddings.csv", index = False)
undersampled_df.head()

Gender Counts:
 gender
female    105
male      105
Name: count, dtype: int64

Split Counts:
 gender  split
female  train    71
        dev      20
        test     14
male    train    58
        test     26
        dev      21
Name: count, dtype: int64


,split,PTSD_severity,PTSD_label,gender,audio,text
0,train,51.0,1,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
1,test,26.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
2,test,49.0,0,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
3,test,19.0,0,male,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
4,dev,21.0,0,female,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...,/home/hice1/mbibars3/scratch/vlm-debiasing/dat...
